In [ ]:
def random_parameters(n_rows, n_columns):
    """This function creates and initializes at random the membership vectors.
       :param n_rows: integer, number of instances of the feature in the dataset
       :param n_columns: integer, number of groups of the feature
       :return param with all the membership vectors (normalized) for the feature """
    import numpy as np

    param = np.random.randint(100, size = (n_rows, n_columns))
    param = param.astype(np.float) #otherwise, we won't be able to work with floats

    #Normalization:
    norms = np.sum(param, axis = 1)
    
    #is the any pair with 2 zeros? if so, change the values
    i = 0
    for n in norms:
        if n==0:
            param[i] = np.array([20, 50])
        i += 1
        
    norms2 = np.sum(param, axis = 1) #we compute the sum again
    i = 0
    for vect in param:
        param[i] = vect/norms2[i] #normalized
        i += 1
        
    return(param)

def random_matrices(n_rows, n_columns, var_values): 
    """It creates and initializes random 3D matrices as the first step of the iterative algorithm given the number of rows and columns, 
    will which coincide with the number of groups of the different features. 
    The access to any value will be possible by following these order: [link_value, alpha, beta or gamma or delta]
    The different weights of these links will also be defined by the var_values (list of these values).
    :param n_rows: integer, in this case, number of groups of conditions
    :param n_columns: integer, in this case, number of groups of a feature (MoA, gene, cell)
    :param var_values: list, it contains the different values a link of condition-feature can have (for ex, [0, 1] for cond-MoA) 
    :return matrix n_rows x n_columns"""
    import numpy as np
    matrix = np.random.randint(100, size = (len(var_values), n_rows, n_columns)) #3D array
    matrix = matrix.astype(np.float) #otherwise, we won't be able to work with floats
    
    ##### Normalization among the 1st dimension:
    norms = np.sum(matrix, axis=0) 
    
    #Is there any pair of zeros? If so, we need to change the values (otherwise, we'll have problems with the division)
    r = 0
    for row in norms:
        c = 0
        for col in row:
            if col==0:
                matrix[:, r, c] = np.array([45, 95]) #any values in here
                
        #Recomputing the sum:
    norms2 = np.sum(matrix, axis=0)
    for link in range(len(var_values)):
        matrix[link] = matrix[link]/norms2 #normalized (the sum will always be 1)
        
    return(matrix)

def aux_functions(data_cond_feature, theta, K, mvector, N_groups, proba_matrix):
    """This function recomputes the auxiliar functions given some parameters.
    :param data_cond_feature: np array, with the data of the condition and the desired feature (moa, gene or cell) 
    :param theta: Series data structure, parameter theta (condition membership vectors)
    :param K: integer, number of condition groups
    :param mvector: Series data structure, parameter of the feature (feature membership vectors)
    :param N_groups: integer, number of feature groups
    :param proba_matrix: np array, probability matrix for the condition and the feature
    :return aux: auxiliar function recalculated
    """
    import numpy as np
    aux = np.zeros((data_cond_feature.shape[0], data_cond_feature.shape[1], K, N_groups))
    for condition in range(data_cond_feature.shape[0]):
        for feature in range(data_cond_feature.shape[1]):
            #at this point, the denominator will be the same for each combination, we'll be adding the values of each numerator
            denominator = 0
            for alpha in range(K): #groups of conditions
                for group in range(N_groups): #groups of the other feature
                    #numerator of omega?
                    this_theta = theta[condition][alpha]
                    this_proba_matrix = proba_matrix[data_cond_feature[condition, feature]][alpha][group]
                    this_mvector = mvector[feature][group]
                    numerator = this_theta*this_proba_matrix*this_mvector
                    aux[condition][feature][alpha][group] = numerator
                    denominator += numerator
            aux[condition][feature] = aux[condition][feature]/denominator 
    return(aux)

def parameter_theta(data_moa, data_gene, data_cell, theta, K, L, T, F, omega, psi, pi, lambda_g, lambda_v):
    """This function recomputes the parameter theta (with the condition membership vectors).
    :param data_moa: np array with the condition-MoA data
    :param data_gene: np array with the condition-gene data
    :param data_cell: np array with the condition-cell data
    :param theta: Series data structure with the parameter theta (condition membership vectors)
    :param K: integer, number of condition groups
    :param L: integer, number of MoA groups
    :param T: integer, number of gene groups
    :param F: integer, number of cell groups
    :param omega: np array with the auxiliar function for condition-MoA
    :param psi: np array with the auxiliar function for condition-gene
    :param pi: np array with the auxiliar function for condition-cell
    :param lambda_g: weight for the gene expression data
    :param lambda_v: weight for the cell viability data
    :return theta: theta recalculated
    """
    
    #Conditions with all the data (including MoAs):
    for condition in range(data_moa.shape[0]):
        for alpha in range(K):
            #condition and alpha fixed
            theta_value = 0
            
            #1st term: omegas
            general_omega = 0
            for moa in range(data_moa.shape[1]):
                local = 0
                for beta in range(L):
                    local += omega[condition][moa][alpha][beta]
                general_omega += local
            
            #2nd term: psis
            general_psi = 0
            for gene in range(data_gene.shape[1]):
                local = 0
                for gamma in range(T):
                    local += psi[condition][gene][alpha][gamma]
                general_psi += local
            
            #3rd term: pis
            general_pi = 0
            for cell in range(data_cell.shape[1]):
                local = 0
                for delta in range(F):
                    local += pi[condition][cell][alpha][delta]
                general_pi += local
            
            #Value of theta for condition and alpha:
            num_theta = general_omega + lambda_g*general_psi + lambda_v*general_pi
            den_theta = data_moa.shape[1] + lambda_g*data_gene.shape[1] + lambda_v*data_cell.shape[1]
            theta[condition][alpha] = num_theta/den_theta
            
    #Conditions without MoAs data:
    for condition in range(data_moa.shape[0], data_gene.shape[0]): #difference between the whole dataset and the one with MoAs
        for alpha in range(K):
            #condition and alpha fixed
            theta_value = 0
            
            #1st term: omegas --> NOT IN HERE
            
            #2nd term: psis
            general_psi = 0
            for gene in range(data_gene.shape[1]):
                local = 0
                for gamma in range(T):
                    local += psi[condition][gene][alpha][gamma]
                general_psi += local
            
            #3rd term: pis
            general_pi = 0
            for cell in range(data_cell.shape[1]):
                local = 0
                for delta in range(F):
                    local += pi[condition][cell][alpha][delta]
                general_pi += local
                
            #Value of theta for condition and alpha:
            num_theta = lambda_g*general_psi + lambda_v*general_pi
            den_theta = lambda_g*data_gene.shape[1] + lambda_v*data_cell.shape[1]
                #we don't take into account the cardinality of the set with MoAs associated!!!
            theta[condition][alpha] = num_theta/den_theta
        
    return(theta)

def parameter_eta(data, eta, K, L, omega):
    """This function recomputes the parameter eta (with the MoA membership vectors).
    :param data: np array with the condition-MoA data
    :param eta: Series data structure with the parameter eta (MoA membership vectors)
    :param K: integer, number of condition groups
    :param L: integer, number of MoA groups
    :param omega: np array with the auxiliar function for condition-MoA
    :return eta: eta recalculated
    """
    for moa in range(data.shape[1]):
        for beta in range(L):
            #moa and beta fixed
            general = 0
            for condition in range(data.shape[0]):
                local = 0
                for alpha in range(K):
                    local += omega[condition][moa][alpha][beta]
                general += local
            eta[moa][beta] = general/data.shape[0]
    return(eta)

def parameter_phi(data, phi, K, T, psi):
    """This function recomputes the parameter phi (with the gene membership vectors).
    :param data: np array with the condition-gene data
    :param phi: Series data structure with the parameter phi (gene membership vectors)
    :param K: integer, number of condition groups
    :param T: integer, number of gene groups
    :param psi: np array with the auxiliar function for condition-gene
    :return phi: phi recalculated
    """
    for gene in range(data.shape[1]):
        for gamma in range(T):
            #gene and gamma fixed
            general = 0
            for condition in range(data.shape[0]):
                local = 0
                for alpha in range(K):
                    local += psi[condition][gene][alpha][gamma]
                general += local
            phi[gene][gamma] = general/data.shape[0]
    return(phi)

def parameter_xi(data, xi, K, F, pi):
    """This function recomputes the parameter xi (with the cell membership vectors).
    :param data: np array with the condition-cell data
    :param xi: Series data structure with the parameter xi (cell membership vectors)
    :param K: integer, number of condition groups
    :param F: integer, number of cell groups
    :param pi: np array with the auxiliar function for condition-cell
    :return xi: xi recalculated
    """
    for cell in range(data.shape[1]):
        for delta in range(F):
            #cell and delta fixed
            general = 0
            for condition in range(data.shape[0]):
                local = 0
                for alpha in range(K):
                    local += pi[condition][cell][alpha][delta]
                general += local
            xi[cell][delta] = general/data.shape[0]
    return(xi)

def parameter_p_matrix(data, p_matrix, K, L, omega):
    """This function recomputes the probability matrix p (condition-MoA).
    :param data: np array with the condition-MoA data
    :param p_matrix: np array with the probability matrix p 
    :param K: integer, number of condition groups
    :param L: integer, number of MoA groups
    :param omega: np array with the auxiliar function for condition-MoA
    :return p_matrix: p_matrix recalculated
    """
    for alpha in range(K):
        for beta in range(L):
            #alpha and beta fixed
            numer = [0, 0] #2 values for the 2 possible links: 0 or 1
            denom = 0
            for condition in range(data.shape[0]):
                for moa in range(data.shape[1]):
                    link = data[condition, moa] #0 or 1
                    om = omega[condition][moa][alpha][beta]
                    numer[link] += om
                    denom += om   
            p_matrix[0][alpha][beta] = numer[0]/denom
            p_matrix[1][alpha][beta] = numer[1]/denom
    return(p_matrix)

def parameter_q_matrix(data, q_matrix, K, T, psi):
    """This function recomputes the probability matrix q (condition-gene).
    :param data: np array with the condition-gene data
    :param q_matrix: np array with the probability matrix q 
    :param K: integer, number of condition groups
    :param T: integer, number of gene groups
    :param psi: np array with the auxiliar function for condition-gene
    :return q_matrix: q_matrix recalculated
    """
    for alpha in range(K):
        for gamma in range(T):
            #alpha and gamma fixed
            numer = [0, 0, 0] #3 possible links: -1, 0, 1
            denom = 0
            for condition in range(data.shape[0]):
                for gene in range(data.shape[1]):
                    link = data[condition, gene]
                    ps = psi[condition][gene][alpha][gamma]
                    numer[link] += ps
                    denom += ps
            q_matrix[0][alpha][gamma] = numer[0]/denom        
            q_matrix[1][alpha][gamma] = numer[1]/denom
            q_matrix[2][alpha][gamma] = numer[2]/denom
    return(q_matrix)

def parameter_s_matrix(data, s_matrix, K, F, pi):
    """This function recomputes the probability matrix s (condition-cell).
    :param data: np array with the condition-cell data
    :param s_matrix: np array with the probability matrix s 
    :param K: integer, number of condition groups
    :param F: integer, number of cell groups
    :param pi: np array with the auxiliar function for condition-cell
    :return s_matrix: s_matrix recalculated
    """
    for alpha in range(K):
        for delta in range(F):
            #alpha and delta fixed
            numer = [0, 0] #2 possible links: 0 or 1
            denom = 0
            for condition in range(data.shape[0]):
                for cell in range(data.shape[1]):
                    link = data[condition, cell]
                    p = pi[condition][cell][alpha][delta]
                    numer[link] += p
                    denom += p
            s_matrix[0][alpha][delta] = numer[0]/denom
            s_matrix[1][alpha][delta] = numer[1]/denom
    return(s_matrix)

def check_log_posterior(data_moa, data_gene, data_cell, theta, eta, phi, xi, p_matrix, 
                                 q_matrix, s_matrix, K, L, T, F, lambda_g, lambda_v):
    """This function computes the log posterior value, which gives different weights to the different terms 
    (so as to control the importance of the data associated with gene expression and cell viability).
    :param data_moa: np array with the condition-MoA data
    :param data_gene: np array with the condition-gene data
    :param data_cell: np array with the condition-cell data
    :param theta: Series data structure with the parameter theta (condition membership vectors)
    :param eta: Series data structure with the parameter eta (MoA membership vectors)
    :param phi: Series data structure with the parameter phi (gene membership vectors)
    :param xi: Series data structure with the parameter xi (cell membership vectors)
    :param p_matrix: np array with the probability matrix p
    :param q_matrix: np array with the probability matrix q
    :param s_matrix: np array with the probability matrix s
    :param K: integer, number of condition groups
    :param L: integer, number of MoA groups
    :param T: integer, number of gene groups
    :param F: integer, number of cell groups
    :param lambda_g: weight for the gene expression data
    :param lambda_v: weight for the cell viability data
    :return logposterior: value of this score
    """
    import math

    log_posterior_moa = 0
    log_posterior_gene = 0
    log_posterior_cell = 0
    
    #observed MoA:
    for condition in range(data_moa.shape[0]):
        for moa in range(data_moa.shape[1]):
            adding_moa = 0
            real_link = data_moa[condition, moa] #real value of the link between the current condition and moa
            for alpha in range(K):
                for beta in range(L):
                    proba = theta[condition][alpha]*eta[moa][beta]*p_matrix[real_link][alpha][beta]
                    adding_moa += proba
            log_posterior_moa += math.log(adding_moa)
            
    #observed and non observed MoA:
    for condition in range(data_cell.shape[0]): #or df_cond_gene.index (they are the same)
        for cell in range(data_cell.shape[1]):
            adding_cell = 0
            real_link = data_cell[condition, cell] #real value of the link between the current condition and cell
            for alpha in range(K):
                for delta in range(F):
                    proba = theta[condition][alpha]*xi[cell][delta]*s_matrix[real_link][alpha][delta]
                    adding_cell += proba
            log_posterior_cell += math.log(adding_cell)
            
        for gene in range(data_gene.shape[1]):
            adding_gene = 0
            real_link = data_gene[condition, gene] #real value of the link between the current condition and gene
            for alpha in range(K):
                for gamma in range(T):
                    proba = theta[condition][alpha]*phi[gene][gamma]*q_matrix[real_link][alpha][gamma]
                    adding_gene += proba
            log_posterior_gene += math.log(adding_gene)
            
    log_posterior = log_posterior_moa + lambda_g*log_posterior_gene + lambda_v*log_posterior_cell
    
    return(log_posterior)